In [1]:
import numpy as np

from transforms84.distances import Haversine
from transforms84.geodetic_systems import WGS84
from transforms84.helpers import (
    DDM2RRM,
    RRM2DDM,
    deg_angular_difference,
    deg_angular_differences,
)
from transforms84.transforms import ECEF2ENU, ENU2AER, geodetic2ECEF

## Coordinate Transformations

In [2]:
rrm_local = DDM2RRM(np.array([[30], [31], [0]], dtype=np.float64))
rrm_target = DDM2RRM(np.array([[31], [32], [0]], dtype=np.float64))

In [3]:
RRM2DDM(rrm_target)

array([[31.],
       [32.],
       [ 0.]])

In [4]:
rrm_local2target = ENU2AER(
    ECEF2ENU(rrm_local, geodetic2ECEF(rrm_target, WGS84.a, WGS84.b), WGS84.a, WGS84.b)
)
ddm_local2target = RRM2DDM(rrm_local2target)
ddm_local2target

array([[ 4.06379074e+01],
       [-6.60007585e-01],
       [ 1.46643956e+05]])

In [5]:
np.tile(rrm_local, 3).T.reshape((-1, 3, 1))

array([[[0.52359878],
        [0.54105207],
        [0.        ]],

       [[0.52359878],
        [0.54105207],
        [0.        ]],

       [[0.52359878],
        [0.54105207],
        [0.        ]]])

In [6]:
rrm_targets = np.ascontiguousarray(np.tile(rrm_target, 3).T.reshape((-1, 3, 1)))
rrm_locals = np.ascontiguousarray(
    np.tile(rrm_local, rrm_targets.shape[0]).T.reshape((-1, 3, 1))
)
RRM2DDM(rrm_targets), rrm_targets

(array([[[31.],
         [32.],
         [ 0.]],
 
        [[31.],
         [32.],
         [ 0.]],
 
        [[31.],
         [32.],
         [ 0.]]]),
 array([[[0.54105207],
         [0.55850536],
         [0.        ]],
 
        [[0.54105207],
         [0.55850536],
         [0.        ]],
 
        [[0.54105207],
         [0.55850536],
         [0.        ]]]))

In [7]:
ddm_local2targets = RRM2DDM(
    ENU2AER(
        ECEF2ENU(
            rrm_locals, geodetic2ECEF(rrm_targets, WGS84.a, WGS84.b), WGS84.a, WGS84.b
        )
    )
)
assert np.all(np.unique(ddm_local2targets, axis=0)[0] == ddm_local2target)
ddm_local2targets

array([[[ 4.06379074e+01],
        [-6.60007585e-01],
        [ 1.46643956e+05]],

       [[ 4.06379074e+01],
        [-6.60007585e-01],
        [ 1.46643956e+05]],

       [[ 4.06379074e+01],
        [-6.60007585e-01],
        [ 1.46643956e+05]]])

## Distance Calculation
Compare the above range to the Haversine method:

In [8]:
m_distance = Haversine(rrm_local, rrm_target, WGS84.mean_radius)
m_distance, m_distance - ddm_local2target[2, 0]

(array([146775.8833037]), array([131.92690735]))

In [9]:
m_distances1 = Haversine(rrm_locals, rrm_targets, WGS84.mean_radius)
m_distances1

array([146775.8833037, 146775.8833037, 146775.8833037])

In [10]:
m_distances2 = Haversine(
    np.ascontiguousarray(np.tile(rrm_local, 10).T.reshape((-1, 3, 1))),
    np.ascontiguousarray(np.tile(rrm_target, 10).T.reshape((-1, 3, 1))),
    (2 * 6378137.0 + 6356752.314245) / 3,
)
m_distances2

array([146775.8833037, 146775.8833037, 146775.8833037, 146775.8833037,
       146775.8833037, 146775.8833037, 146775.8833037, 146775.8833037,
       146775.8833037, 146775.8833037])

In [11]:
m_distance == np.unique(m_distances1)[0] == np.unique(m_distances2)[0]

np.True_

## Misc. Functions

In [12]:
(
    deg_angular_difference(50, 70, True),
    deg_angular_difference(70, 50, True),
    deg_angular_difference(70, 50, False),
)

(20.0, 20.0, 340.0)

In [13]:
(
    deg_angular_differences(
        np.array([50, 50], dtype=np.float32), np.array([70, 70], dtype=np.float32), True
    ),
    deg_angular_differences(
        np.array([70, 70], dtype=np.float32), np.array([50, 50], dtype=np.float32), True
    ),
    deg_angular_differences(
        np.array([70, 70], dtype=np.float32),
        np.array([50, 50], dtype=np.float32),
        False,
    ),
)

(array([20., 20.], dtype=float32),
 array([20., 20.], dtype=float32),
 array([340., 340.], dtype=float32))